# Setup

## Pip install

In [ ]:
# Don't forget to restart runtime after installing

%pip install "labelbox[data]" --quiet
%pip install -U kaleido  --quiet # for saving the still figures
%pip install poppler-utils   # for exporting to .eps extension
%pip install plotly==5.7.0.    # need 5.7.0, not 5.5, so I can use ticklabelstep argument
# %pip freeze
# %pip freeze | grep matplotlib  # get version

     |████████████████████████████████| 167 kB 5.2 MB/s 
     |████████████████████████████████| 11.1 MB 10.0 MB/s 
     |████████████████████████████████| 6.3 MB 41.3 MB/s 
     |████████████████████████████████| 79.9 MB 130 kB/s 


## Base imports


In [ ]:
import os
import sys
print(sys.version)
import json
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import skimage
import skimage.io
#from PIL import Image
import PIL
import PIL.Image
import requests

import labelbox
#from labelbox.data.annotation_types import Geometry

import IPython.display
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

In [ ]:

notebook_filename = requests.get("http://172.28.0.2:9000/api/sessions").json()[0]["name"]

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  display(IPython.display.Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)


#@markdown ### func `def get_path_to_save(...):`
def get_path_to_save(plot_props:dict=None, file_prefix="", save_filename:str=None, save_in_subfolder:str=None, extension="jpg", create_folder_if_necessary=True):
    """
    Code created myself (Rahul Yerrabelli)
    """
    replace_characters = {
        "$": "",
        "\\frac":"",
        "\\mathrm":"",
        "\\left(":"(",
        "\\right)":")",
        "\\left[":"[",
        "\\right]":"]",
        "\\": "",
        "/":"-",
        "{": "(",
        "}": ")",
        "<":"",
        ">":"",
        "?":"",
        "_":"",
        "^":"",
        "*":"",
        "!":"",
        ":":"-",
        "|":"-",
        ".":"_",
    }

    # define save_filename based on plot_props
    if save_filename is None:
        save_filename = "unnamed"

    #save_path = f"../outputs/{notebook_filename.split('.',1)[0]}"
    save_path = [
                 "outputs",
                f"{notebook_filename.split('.',1)[0]}",
                ]
    if save_in_subfolder is not None:
        if isinstance(save_in_subfolder, (list, tuple, set, np.ndarray) ):
            save_path.append(**save_in_subfolder)
        else:  # should be a string then
            save_path.append(save_in_subfolder)
    save_path = os.path.join(*save_path)

    if not os.path.exists(save_path) and create_folder_if_necessary:
        os.makedirs(save_path)
    return os.path.join(save_path, file_prefix+save_filename+"."+extension)
    #plt.savefig(os.path.join(save_path, save_filename+"."+extension))



In [ ]:
#@title ## Mount google drive and import my code

mountpoint_folder_name = "drive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/SpeculumAnalysis" #@param {type:"string"}
#project_path_within_drive = "UIUC ECs/Rahul_Ashkhan_Projects/SpeculumProjects_Shared/Analysis" #@param {type:"string"}
project_path_full = os.path.join("/content/",mountpoint_folder_name,
                        "MyDrive",project_path_within_drive)

%cd {project_path_full}

In [ ]:
try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name

    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

# Skip ahead from loaded code

In [ ]:
speculum_df_raw = pd.read_pickle("data/02_intermediate/speculum_df_raw"+".pkl")
speculum_df_notfailed = pd.read_pickle("data/02_intermediate/speculum_df_notfailed"+".pkl")

labels_df = pd.read_csv("data/02_intermediate/labels_df.csv", index_col=0)
#with open("data/02_intermediate/label_from_id_dict"+".json", "r") as infile:
#    label_from_id_dict = json.load(infile)
    
df_long = pd.read_pickle(  "data/03_processed/combined_df_long.pkl")
df_wide = pd.read_pickle(  "data/03_processed/combined_df_wide.pkl")
df_wide_flat = pd.read_pickle(  "data/03_processed/combined_df_wide_flat.pkl")

df_agg_long = pd.read_pickle("data/04_aggregated/combined_df_agg_long.pkl")
df_agg_long_flat = pd.read_pickle("data/04_aggregated/combined_df_agg_long_flat.pkl")

df_multiindex = pd.read_pickle("data/03_processed/combined_df_multiindex"+".pkl")


# Set up for displaying

## Setup dicts and helper functions

In [ ]:
category_orders={
    #"Size": ["S", "M", "L","Unspecified","None"],
    #"Size": ["S","Small", "M", "Medium", "L", "Large","Unspecified","None"],
    "Size": ["Small", "Medium", "Large","Unspecified","None", "S", "M", "L"],  # change the order of S vs Small etc changes the color
    "Material":["Nitrile","Vinyl","Trojan", "Lifestyle", "Durex", "Skyn","None"],
    "Material Type":["Glove","Condom","None"],
    "Method":["Middle","Two","Palm","Middle finger","Two fingers","Palm","Precut","None"],
    "Speculum Type":["White","Green"]
    }
labels = {
    "Trial":"Trial #",
    "wd_rel":"Relative Obstruction",
    "wd_rel.mean":"Mean Relative Obstruction (S.E.)", 
    "mmHg":"Pressure (mmHg)", 
    "Material":"Material", "Material Type":"Material Type"
    }


def criteria_to_str(criteria:dict) -> str:
    return ", ".join([f"{labels.get(key) or key}={val}" for key,val in criteria.items()])


def filter_by_criteria(criteria:dict, starting_df:pd.DataFrame) -> pd.DataFrame:
    #df_sampled = df_agg_long_flat.loc[ np.all([df_agg_long[arg]==val for arg, val in criteria.items()], axis=0) ]
    #df_sampled = df_agg_long_flat.loc[ np.all([ (type(val)!=list and df_agg_long[arg]==val ) or np.in1d(df_agg_long[arg],val)  for arg, val in criteria.items()], axis=0) ]
    #starting_df.loc[ np.all([ (type(val)!=list and starting_df[arg]==val ) or np.in1d(starting_df[arg],val)  for arg, val in criteria.items()], axis=0) ]
    conditions = []
    for arg, val in criteria.items():
        if hasattr(val,"__iter__") and not isinstance(val,str):
            conditions.append( np.in1d(starting_df[arg],val) )
        else:
            conditions.append( starting_df[arg]==val )
    return starting_df.loc[ np.all(conditions, axis=0) ]

## Setup plotly figure saving

In [ ]:
default_plotly_save_scale = 4
def save_plotly_figure(fig, file_name:str, animated=False, scale=None, save_in_subfolder:str=None, extensions=None):
    """
    - for saving plotly.express figures only - not for matplotlib
    - fig is of type plotly.graph_objs._figure.Figure,
    - Requires kaleido installation for the static (non-animated) images
    """    
    if scale is None:
        scale = default_plotly_save_scale
    if extensions is None:
        extensions = ["html"]
        if not animated:
            # options = ['png', 'jpg', 'jpeg', 'webp', 'svg', 'pdf', 'eps', 'json']
            extensions += ["eps","png","pdf"]

    for extension in extensions:
        try:
            if extension in ["htm","html"]:
                #fig.update_layout(title=dict(visible=False))
                fig.write_html( get_path_to_save(save_filename=file_name, save_in_subfolder=save_in_subfolder, extension=extension), 
                    full_html=False,
                    include_plotlyjs="directory" )
            else:
                #if extension == "png":
                #    fig.update_layout(title=dict(visible=False))
                fig.write_image(get_path_to_save(save_filename=file_name, save_in_subfolder=save_in_subfolder, extension=extension), scale=scale)
        except ValueError as exc:
            import traceback
            #traceback.print_exception()

#col_options = {col_name:pd.unique(df_long[col_name]).tolist() for col_name in consistent_cols}
#display(col_options)

# Plotting

## Plot aggregates across trials

#### Setup for plotting aggregates

In [ ]:
def customize_figure(fig, width=640, height=360, by_mmHg=True, brs=1) -> dict:
    """ - for plotly figures only. """
    
    if by_mmHg:
        fig.update_xaxes( #tickprefix="At ",   # Dr. WJ and Ashkhan didn't like it
                         ticksuffix="mmHg", showtickprefix="all", showticksuffix="all", tickfont=dict(size=16),
                        mirror=True, linewidth=2, 
                        title=dict(text="Applied Circumferential Pressure", font=dict(size=20, family="Arial Black")),
                        )
        fig.update_yaxes(tickformat=".0%", tickwidth=2,  nticks=21, ticklabelstep=4,
                        mirror="ticks", linewidth=2, range=(0,1), 
                        title=dict(text="Obstruction of<br>Field of View (S.E.)",font=dict(size=18, family="Arial Black")), 
                        #title=dict(text="Width Obstructed of<br>Field of View (S.E.)",font=dict(size=18, family="Arial Black")), 
                        showgrid=True, gridcolor="#DDD", 
                        showspikes=True, spikemode="across", spikethickness=2, spikedash="solid", # ticklabelposition="inside top",
                        )
    #fig.update_traces(textangle=0, textposition="outside", cliponaxis=False)
    fig.update_layout(
        font=dict(
            family="Arial",
            size=16,
            color="black",
        ),
        title={
            "y":1,
            "x":0.5,
            "xanchor": "center",
            "yanchor": "top",
            "font":dict(size=16)
        }, 
        width=width, height=height,
        margin=dict(l=20, r=20, t=20, b=20),
        legend=dict(
            title={"font_family": "Arial Black",},
            yanchor="middle",
            y=0.5,
            xanchor="center",
            x=0.08,
            #bgcolor="LightSteelBlue",
            bordercolor="Black", #font_size=16,
            borderwidth=2,
        ), 
        bargap=0.05, bargroupgap=0.0,
        dragmode="drawopenpath",
        newshape_line_color="cyan",
    )

    fig.update_traces(textfont_size=16, textangle=0, textfont_color="#FFF", 
                      textposition="inside", cliponaxis=False, #textfont_family="Courier",
                      marker_line_color="#000", marker_line_width=2
                    )
    if by_mmHg:
        fig.update_traces(texttemplate=[None]+[" "+("<br>"*brs)+"<b>%{y:.1%}</b>"]*5,)

    config = {
        "toImageButtonOptions" : {
            "format": "png", # one of png, svg, jpeg, webp
            "filename": 'custom_image',
            "scale": default_plotly_save_scale # Multiply title/legend/axis/canvas sizes by this factor
        },
        "modeBarButtonsToAdd": ["drawline","drawopenpath","drawclosedpath","drawcircle","drawrect","eraseshape"]
    }

    return config






<IPython.core.display.Javascript object>

### Gloves

#### Glove sizes

In [ ]:
#criteria = {"Material":["Nitrile","None"], "Method":["Middle","None"]}
criteria = {"Material":"Nitrile", "Method":"Middle"}
varying = "Size"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
df_sampled["Size"] = df_sampled["Size"].replace({"S":"Small", "M":"Medium", "L":"Large"})

fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", #error_y_minus=[0]*18, 
             color=varying, pattern_shape=varying, 
             color_discrete_sequence=px.colors.qualitative.Safe, pattern_shape_sequence=["/", "+", "\\"], 
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels=labels, template="simple_white", 
             )
#fig.update_traces(hovertemplate="""%{x}""") #
config = customize_figure(fig, width=1100, height=300)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.show(config=config)
fig.update_layout(title=dict(text=""))
save_plotly_figure(fig, file_name=f"Fig 4- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Glove material

In [ ]:
#criteria = {"Size":["M","None"], "Method":["Middle","None"]}
criteria = {"Size":"M", "Method":"Middle"}

varying = "Material"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", 
             color=varying, pattern_shape=varying, 
             color_discrete_sequence=px.colors.qualitative.Safe[6:-1:], pattern_shape_sequence=["x", "+", "\\"], 
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method"],
             title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels={**labels,"Material":"Glove<br>Material"}, template="simple_white", 
             )

config = customize_figure(fig, width=1100, height=300)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.show(config=config)
fig.update_layout(title=dict(text=""))
save_plotly_figure(fig, file_name=f"Fig 5- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

#### Glove method

In [ ]:
#criteria = {"Size":["M","None"], "Material":["Nitrile","None"]}
criteria = {"Size":"M", "Material":"Nitrile"}

varying = "Method"

df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
df_sampled["Method"] = df_sampled["Method"].replace({"Middle":"Middle finger","Two":"Two fingers"})
fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", 
             color=varying, pattern_shape=varying, 
             color_discrete_sequence=px.colors.qualitative.D3, pattern_shape_sequence=["x", "+", "-"], 
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method","wd_rel.amin","wd_rel.median","wd_rel.amax"],
             #title=f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels=labels, template="simple_white", 
             )

config = customize_figure(fig, width=1100, height=300)

fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )


fig.show(config=config)
fig.update_layout(title=dict(text=""))
save_plotly_figure(fig, file_name=f"Fig 6- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Relative Obstruction of Field of View

Percent Field of View Obstructed

### Plot condoms

In [ ]:
#criteria = {"Material Type":"Condom"}
criteria = {"Material Type":["Condom","None"]}
varying = "Material"

#colors = px.colors.qualitative.Plotly.copy() #[0:4]+["black"]
#colors = ['black']*10
#colors[8] = "black"
#colors = ['#636EFA', '#EF553B',"Black", '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
#colors={"Trojan":"red","gray":"blue","None":"green","Lifestyle":"green","Skyn":"blue"}
#df_sampled = df_agg_long_flat.loc[ np.all([df_agg_long[arg]==val for arg, val in criteria.items()], axis=0) ]
df_sampled = filter_by_criteria(criteria,df_agg_long_flat)
#df_sampled["color"] = df_sampled["Material"].copy().replace(colors)
colors = px.colors.qualitative.Safe #[0:4]+["black"]
fig = px.bar(df_sampled, 
             x="mmHg",y="wd_rel.mean", error_y="wd_rel.sem", 
             color=varying, pattern_shape=varying, 
             color_discrete_sequence= px.colors.qualitative.Safe, pattern_shape_sequence=["|", "-", "\\"], 
             barmode="group", #text=[".1%<br><br> " for a in range(18)],
             hover_data=["Size","Material","Method"],
             title="", #f"Varying {varying} with " + criteria_to_str(criteria), 
             category_orders=category_orders, labels={**labels,"Material":"Condom<br>Brand"}, template="simple_white", 
             )

config = customize_figure(fig, width=1100, height=300)

for idx, trace in enumerate(fig["data"]):
    trace["name"] = trace["name"].split()[-1]

#fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape=""))) if trace.name == "None" else (), )
fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#CCC",pattern=dict(shape=""))) if trace.name == "None" else (), )

fig.update_traces(texttemplate=[None]+[""" <br><br><br><b>%{y:.1%}</b>"""]*5,)

fig.show(config=config)

save_plotly_figure(fig, file_name=f"Fig 7- Across {varying}- " + criteria_to_str(criteria) )

<IPython.core.display.Javascript object>

## Plot vertical heights

In [ ]:
colors={
    "None": "black",
    "<i>Durex</i><br>Condom": "blue",
    "<i>Lifestyle</i><br>Condom": "blue",
    "<i>Skyn</i><br>Condom": "blue",
    "<i>Trojan</i><br>Condom": "blue",
    "Medium<br><i>Vinyl</i><br>Glove": "orange",
    "<i>Large</i><br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>": "orange",
    "<i>Small</i><br>Nitrile<br>Glove": "orange",
    "<i>Small</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
    "<i>Medium</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
}

<IPython.core.display.Javascript object>

In [ ]:
#[key for key in df_sampled["name_formatted"] if key not in colors.keys()]
len(df_sampled["name_formatted"])

<IPython.core.display.Javascript object>

12

In [ ]:
list(names.values())[0:-1:]

<IPython.core.display.Javascript object>

['<i>Small</i><br>Nitrile<br>Glove,<br><i>Palm</i>',
 '<i>Medium</i><br>Nitrile<br>Glove,<br><i>Palm</i>',
 'None',
 '<i>Durex</i><br>Condom',
 '<i>Lifestyle</i><br>Condom',
 '<i>Skyn</i><br>Condom',
 '<i>Trojan</i><br>Condom',
 '<i>Large</i><br>Nitrile<br>Glove',
 'Medium<br>Nitrile<br>Glove',
 '<i>Small</i><br>Nitrile<br>Glove',
 'Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>']

In [ ]:
criteria = {"mmHg":[0,1], "Spec Ang":[3,5]}
varying = "Material"

df_sampled = df_agg_long_flat.loc[ np.all([ (type(val)!=list and df_agg_long[arg]==val ) or np.in1d(df_agg_long[arg],val)  for arg, val in criteria.items()], axis=0) ]
df_sampled = df_sampled.sort_values(["Vertical Height.mean"]).reset_index()
df_sampled["Spec Ang"] = df_sampled["Spec Ang"].astype(str)  # makes discrete color plotting and string concatenation easier
df_sampled["name"] = df_sampled["Size"] + "-" + df_sampled["Material"] + "-"  + df_sampled["Material Type"] + "-"  + df_sampled["Method"] + "-"  + df_sampled["Spec Ang"]

extra_trials = speculum_df_raw.loc[speculum_df_raw["Filename"]=="None"].copy()
extra_trials = extra_trials.drop(extra_trials[extra_trials["Spec Ang"] == 4].index)
extra_trials["Vertical Height.mean"] = extra_trials["Vertical Height"]
extra_trials["Vertical Height.sem"] = None
with_extra = pd.concat([df_sampled,extra_trials])
with_extra = with_extra.drop(columns=[col for col in with_extra if col not in df_sampled.columns])

df_sampled = with_extra
df_sampled["Spec Ang"] = df_sampled["Spec Ang"].astype(str)  # makes discrete color plotting and string concatenation easier
df_sampled["name"] = df_sampled["Size"] + "-" + df_sampled["Material"] + "-"  + df_sampled["Material Type"] + "-"  + df_sampled["Method"] #+ "-"  + df_sampled["Spec Ang"]

names={
    "None-None-None-None-3": "None", #"None<br>(3 clicks)",
    "None-None-None-None-5": "None", #"None<br>(5 clicks)",
    "Unspecified-Durex-Condom-Precut-3": "<i>Durex</i><br>Condom",
    "Unspecified-Lifestyle-Condom-Precut-3": "<i>Lifestyle</i><br>Condom",
    "Unspecified-Skyn-Condom-Precut-3": "<i>Skyn</i><br>Condom",
    "Unspecified-Trojan-Condom-Precut-3": "<i>Trojan</i><br>Condom",
    "M-Vinyl-Glove-Middle-3": "Medium<br><i>Vinyl</i><br>Glove",
    "L-Nitrile-Glove-Middle-5": "<i>Large</i><br>Nitrile<br>Glove",
    "M-Nitrile-Glove-Middle-5": "Medium<br>Nitrile<br>Glove",
    "M-Nitrile-Glove-Two-5": "Medium<br>Nitrile<br>Glove,<br><i>Two-fingers</i>",
    "S-Nitrile-Glove-Middle-5": "<i>Small</i><br>Nitrile<br>Glove"
}
names={
    "S-Nitrile-Glove-Palm":                 "<i>Small</i><br>Nitrile<br>Glove,<br><i>Palm</i>",
    "M-Nitrile-Glove-Palm":                 "<i>Medium</i><br>Nitrile<br>Glove,<br><i>Palm</i>",
    "None-None-None-None":                  "None", #"None<br>(3 clicks)",
    "Unspecified-Durex-Condom-Precut":      "<i>Durex</i><br>Condom",
    "Unspecified-Lifestyle-Condom-Precut":  "<i>Lifestyle</i><br>Condom",
    "Unspecified-Skyn-Condom-Precut":       "<i>Skyn</i><br>Condom",
    "Unspecified-Trojan-Condom-Precut":     "<i>Trojan</i><br>Condom",
    "L-Nitrile-Glove-Middle":               "<i>Large</i><br>Nitrile<br>Glove",
    "M-Nitrile-Glove-Middle":               "Medium<br>Nitrile<br>Glove",
    "S-Nitrile-Glove-Middle":               "<i>Small</i><br>Nitrile<br>Glove",
    "M-Nitrile-Glove-Two":                  "Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>",
    "M-Vinyl-Glove-Middle":                 "Medium<br><i>Vinyl</i><br>Glove",
}
names = {key: value.replace("<i>","").replace("</i>","") for key, value in names.items()}
colors={
    "None": "black",
    "<i>Durex</i><br>Condom": "blue",
    "<i>Lifestyle</i><br>Condom": "blue",
    "<i>Skyn</i><br>Condom": "blue",
    "<i>Trojan</i><br>Condom": "blue",
    "Medium<br><i>Vinyl</i><br>Glove": "orange",
    "<i>Large</i><br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove": "orange",
    "Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>": "orange",
    "<i>Small</i><br>Nitrile<br>Glove": "orange",
    "<i>Small</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
    "<i>Medium</i><br>Nitrile<br>Glove,<br><i>Palm</i>": "orange",
}
df_sampled["name_formatted"] = df_sampled["name"].replace(names, value=None)  # values=None indicates that names has the values in it (i.e is a dict, not a list)
df_sampled["colors"] = df_sampled["name_formatted"].replace(colors, value=None)
#df_sampled["name"] = df_sampled["name_formatted"].replace(names)

fig = px.bar(df_sampled, 
             #x = np.argsort(df_sampled["Vertical Height.mean"]),
             x = "name_formatted",
             y="Vertical Height.mean", error_y="Vertical Height.sem", 
             category_orders={**category_orders, }, 
             labels={**labels,"Spec Ang":"Number of Clicks Open", "Vertical Height.mean":"Initial Height of <br>Speculum Opening"}, 
             template="simple_white", 
             hover_data=["Size","Material","Method","name"], #color = ["gray","gray","red","gray","gray"]
             color="Material Type", 
             facet_col="Spec Ang", # facet_row="Material Type",
             )
fig.update_xaxes(matches=None)
fig.update_traces(texttemplate=""" <br><b>%{y:.2f}<br>cm</b>""", textposition="outside",)

fig.update_xaxes(linewidth=2, #showticklabels=False, nticks=0,
                 title=dict(
                     #text="Speculum-Material Combination",
                     text="Speculum Sheath",
                     font=dict(size=18, family="Arial Black")
                     ),
                 )
fig.update_yaxes(ticksuffix="cm", tickformat=".0f", tickwidth=2, range=(0,6),  nticks=6*2+1, ticklabelstep=2,
                mirror=True, linewidth=2,
                #title=dict(text="Initial Height of <br>Speculum Opening", font=dict(size=18, family="Arial Black")), 
                 title=dict(font=dict(size=18, family="Arial Black")), 
                showgrid=True, gridcolor="#AAA", 
                showspikes=True, spikemode="across", spikethickness=2, spikedash="solid", # ticklabelposition="inside top",
                )

fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

fig.update_layout(showlegend=False)

fig.add_annotation(x=4, y=0.25,
            text="<b>Failed: <i>Broke speculum</i></b>",
            textangle=-90, xanchor="center", yanchor="bottom",
            font=dict(color="#3976af"), bgcolor="white",  # 3976af is blue
            showarrow=False,
            row=1, col=2)
fig.add_annotation(x=5, y=0.25,
            text="<b><i>Failed: <i>Slipped off speculum</i></b>",
            textangle=-90, xanchor="center", yanchor="bottom",
            font=dict(color="#3976af"), bgcolor="white",
            showarrow=False,
            row=1, col=2)
fig.add_annotation(x=6, y=0.25,
            text="<b><i>Failed: <i>Slipped off speculum</i></b>",
            textangle=-90, xanchor="center", yanchor="bottom",
            font=dict(color="#3976af"), bgcolor="white",
            showarrow=False,
            row=1, col=2)

#fig.for_each_trace( lambda trace: trace.update(marker=dict(color="orange")) if "Glove" in trace.name else (), )
#fig.for_each_trace( lambda trace: trace.update(marker=dict(color="#000",opacity=0.33,pattern=dict(shape="")), color="black") if trace.name == "None" else (), )


for index in range(2):
    #fig.layout.annotations[index]["text"] = f"<b>With {[3,5][index]} (of 7) Clicks Open</b>"
    fig.layout.annotations[index]["text"] = f"<b>With {[3,5][index]} Clicks of Angular Adjustment"

config = customize_figure(fig, width=1200, height=500, by_mmHg=False)


fig.show(config=config)
save_plotly_figure(fig, file_name=f"Fig 3- Vertical Height Bar Plot" )

<IPython.core.display.Javascript object>

In [ ]:
#df_sampled[["name","name_formatted"]]
df_sampled

<IPython.core.display.Javascript object>

,index,Speculum Type,Spec Ang,Spec Ht,Size,Material,Material Type,Method,mmHg,wd.mean,...,ht_rel.count_nonzero,Vertical Height.mean,Vertical Height.sem,Vertical Height.std,Vertical Height.amin,Vertical Height.median,Vertical Height.amax,Vertical Height.count_nonzero,name,name_formatted
0,0.0,White,3,0,M,Vinyl,Glove,Middle,0,265.666667,...,0.0,1.116667,0.092796,0.160728,1.00,1.05,1.30,3.0,M-Vinyl-Glove-Middle,Medium<br><i>Vinyl</i><br>Glove
1,55.0,White,5,0,S,Nitrile,Glove,Middle,0,305.333333,...,0.0,2.733333,0.016667,0.028868,2.70,2.75,2.75,3.0,S-Nitrile-Glove-Middle,<i>Small</i><br>Nitrile<br>Glove
2,42.0,White,5,0,M,Nitrile,Glove,Middle,0,253.666667,...,0.0,2.866667,0.166667,0.288675,2.70,2.70,3.20,3.0,M-Nitrile-Glove-Middle,Medium<br>Nitrile<br>Glove
3,36.0,White,5,0,L,Nitrile,Glove,Middle,0,269.666667,...,0.0,3.133333,0.066667,0.115470,3.00,3.20,3.20,3.0,L-Nitrile-Glove-Middle,<i>Large</i><br>Nitrile<br>Glove
4,12.0,White,3,0,Unspecified,Durex,Condom,Precut,0,248.666667,...,0.0,3.566667,0.016667,0.028868,3.55,3.55,3.60,3.0,Unspecified-Durex-Condom-Precut,<i>Durex</i><br>Condom
5,30.0,White,3,0,Unspecified,Trojan,Condom,Precut,0,243.666667,...,0.0,3.600000,0.050000,0.086603,3.55,3.55,3.70,3.0,Unspecified-Trojan-Condom-Precut,<i>Trojan</i><br>Condom
6,18.0,White,3,0,Unspecified,Lifestyle,Condom,Precut,0,239.333333,...,0.0,3.633333,0.016667,0.028868,3.60,3.65,3.65,3.0,Unspecified-Lifestyle-Condom-Precut,<i>Lifestyle</i><br>Condom
7,24.0,White,3,0,Unspecified,Skyn,Condom,Precut,0,252.666667,...,0.0,3.666667,0.044096,0.076376,3.60,3.65,3.75,3.0,Unspecified-Skyn-Condom-Precut,<i>Skyn</i><br>Condom
8,6.0,White,3,0,None,None,None,None,0,291.333333,...,0.0,4.100000,0.000000,0.000000,4.10,4.10,4.10,3.0,None-None-None-None,None
9,48.0,White,5,0,M,Nitrile,Glove,Two,0,216.666667,...,0.0,5.100000,0.057735,0.100000,5.00,5.10,5.20,3.0,M-Nitrile-Glove-Two,"Medium<br>Nitrile<br>Glove,<br><i>2 fingers</i>"
